In [1]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from load_dataset import *
import os
print(os.getcwd()) 

/home/jovyan/work/nbs


In [2]:
type0_df, type1_df, type2_df, type3_df, type4_df, type5_df, type6_df=load_data()

20000   20000   20000   2747   9493   17367   20510


In [5]:
training_forest_data, validation_forest_data=load_training_and_validation_data(type0_df, type1_df, type2_df, type3_df, type4_df, type5_df, type6_df)

total dataset number:  110117
training_rows_number:  77251  validation_rows_number: 32866


In [6]:
feature_cols=get_features_title()

In [7]:
training_data_final,validation_data_final=transform_data(training_forest_data, validation_forest_data, feature_cols)

final training data:
+-------------------------------------------------------------------------------------------------------+-----+
|features                                                                                               |label|
+-------------------------------------------------------------------------------------------------------+-----+
|(54,[0,1,2,3,4,5,6,7,8,9,10,52],[3388.0,292.0,11.0,395.0,12.0,2910.0,189.0,238.0,189.0,2060.0,1.0,1.0])|6    |
|(54,[0,1,2,3,5,6,7,8,9,13,23],[2339.0,348.0,17.0,30.0,446.0,186.0,210.0,161.0,853.0,1.0,1.0])          |2    |
|(54,[0,1,2,3,4,5,6,7,8,9,13,23],[2151.0,342.0,27.0,309.0,146.0,350.0,154.0,189.0,166.0,518.0,1.0,1.0]) |2    |
|(54,[0,1,2,3,4,5,6,7,8,9,13,23],[2028.0,27.0,24.0,30.0,3.0,331.0,202.0,180.0,104.0,745.0,1.0,1.0])     |2    |
|(54,[0,1,2,3,4,5,6,7,8,9,10,36],[3244.0,57.0,7.0,531.0,23.0,3776.0,225.0,226.0,136.0,3056.0,1.0,1.0])  |0    |
|(54,[0,1,2,3,4,5,6,7,8,9,13,19],[2511.0,108.0,16.0,108.0,28.0,360.0,246.0,220.0,98

In [10]:
layers = [54,54,70,30,10,7, 7]
mp_trainer = MultilayerPerceptronClassifier(maxIter=500, layers=layers, blockSize=128, seed=1234)
model = mp_trainer.fit(training_data_final)
print("The model is trained!")

The model is trained!


In [13]:
result=model.transform(validation_data_final)
predictionAndLabels = result.select("prediction","label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))
prediction_final=result.select("prediction","label")
prediction_final.show(truncate=False,n=20)

Test set accuracy = 0.5835513904947363
+----------+-----+
|prediction|label|
+----------+-----+
|2.0       |2    |
|6.0       |6    |
|2.0       |3    |
|6.0       |4    |
|5.0       |5    |
|6.0       |4    |
|1.0       |1    |
|6.0       |0    |
|6.0       |6    |
|1.0       |0    |
|6.0       |0    |
|1.0       |1    |
|6.0       |6    |
|2.0       |2    |
|2.0       |2    |
|2.0       |4    |
|5.0       |5    |
|2.0       |3    |
|6.0       |6    |
|1.0       |1    |
+----------+-----+
only showing top 20 rows



In [ ]:
#[54,5,6,7]0.6476 90000 dataset iter=100
#[54, 50, 30, 10, 7] 0.6907 90000 dataset iter=300
#[54, 50, 30, 20,10, 7] 0.6857 120000 dataset iter=300
#[54, 100, 50, 7] 0.6965 120000 dataset iter=300

#[54, 54,30,20,7, 7] 0.5250075872534142 iter=300
#[54,54,40,30,20,10, 7] 0.5433990895295903 iter =300
#[54,54,70,30,10, 7] 0.6101062215477997 iter=500